In [1]:
%run utils.ipynb

In [2]:
warnings.filterwarnings('ignore')

In [3]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [4]:
df=pd.read_csv("../data/train.csv")

#drop some useless columns
cols_to_drop=["Id","PID"]
df=df.drop(columns=cols_to_drop)

#drop outliers identified in part 1
df=df[df['Gr Liv Area']<=4500]
df=df[df['SalePrice']>np.expm1(10)]

df_copy=df.copy()

Let's look at the rows which have null, and the numbers

In [5]:
df_null=df.isnull().sum().sort_values(ascending=False).loc[lambda x: x>0]
df_null

Pool QC           2039
Misc Feature      1983
Alley             1907
Fence             1647
Fireplace Qu       999
Lot Frontage       330
Garage Finish      114
Garage Qual        114
Garage Yr Blt      114
Garage Cond        114
Garage Type        113
Bsmt Exposure       57
BsmtFin Type 2      55
Bsmt Cond           54
Bsmt Qual           54
BsmtFin Type 1      54
Mas Vnr Area        22
Mas Vnr Type        22
Bsmt Full Bath       2
Bsmt Half Bath       2
Garage Area          1
Garage Cars          1
Total Bsmt SF        1
Bsmt Unf SF          1
BsmtFin SF 2         1
BsmtFin SF 1         1
dtype: int64

We will impute these columns one way or another.

### Missing values

In [6]:
cols_to_impute_with_none=["Pool QC","Misc Feature","Alley","Fence","Fireplace Qu",
                          "Garage Finish","Garage Qual","Garage Cond","Garage Type",
                         "Bsmt Exposure","BsmtFin Type 2","Bsmt Cond","Bsmt Qual","BsmtFin Type 1",
                         "Mas Vnr Type"]

cols_to_impute_with_zero=["Garage Yr Blt","Mas Vnr Area","Bsmt Full Bath","Bsmt Half Bath","Garage Area",
                         "Garage Cars","Total Bsmt SF","Bsmt Unf SF","BsmtFin SF 2","BsmtFin SF 1"]

cols_to_impute_with_mode=["Electrical"]

Ok! everything looks good now.

### We will change ordinal features to numerical

### We will one hot encode categorical features

Let's take a look at the correlation heatmap

Not very useful, so let's just ignore for now.

From the 7 primary variables we had from part 1,  
`Overall Qual`, `Gr Liv Area`, `Garage Cars`, `Year Built`, `Total Bsmt SF`, `Year Remod/Add`, `Full Bath`,   
we now have a few more:  
`Exter Qual`, `Kitchen Qual`, `Bsmt Qual`, `Garage Finish`, `Foundation_PConc`,`Fireplace Qu`, `Heating QC`.

### Let's run a simple Linear Regression using all the numerical features

### Set up model

In [7]:
#retrieve df_train from backup
df=df_copy.copy()

#set up X and Y
X=df.drop(columns="SalePrice")
y=df["SalePrice"]

#make a backup copy
X_copy=X.copy()
y_copy=y.copy()

#### Set up a `TransformedTargetRegressor` to handle the log / exponential transformation of the label  
Uses Pipeline to perform preprocessing and prediction

In [8]:
#Create a regressor to dynamically creates a pipeline
def create_regressor(model,**kwargs):
    '''
    model: estimator instance
    return: TransformedTargetRegressor
    '''
    pipeline=Pipeline(steps=[
        ('selector',FeatureSelector(**kwargs)),
        ('simple_imputer',SimpleImputer(strategy="constant",fill_value=0)),
#         ('standard_impute',StandardImpute(none=cols_to_impute_with_none,
#                                           zero=cols_to_impute_with_zero,
#                                           mode=cols_to_impute_with_mode)),
#         ('lot_frontage_impute',LotFrontageImpute()),
#         ('ordinal_to_numerical',OrdinalToNumeric()),
#         ('numerical_scaling',Scaling()) reserved for scaling
#         ('onehotencode',OneHotEncode()),
#         ('selector',FeatureSelector(features_to_keep)),
        
        
#         ('categorical_numerical_split',Categorical_Numerical_Split),
#         ('align_train_and_predict',AlignTrainPredict()),
        ('passthrough',Passthrough()),
        ('model',model)
    ])
    
#     display(pipeline)
    
    return TransformedTargetRegressor(regressor=pipeline,
                                    func=np.log1p,
                                    inverse_func=np.expm1)

In [9]:
X=X_copy.copy()

lr=create_regressor(LinearRegression(),by="numerical")

# np.sqrt(-cross_val_score(lr,X,y,cv=5,scoring='neg_mean_squared_error')).mean()

In [10]:
lr.fit(X,y)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=Pipeline(steps=[('selector',
                                                      FeatureSelector(by='numerical')),
                                                     ('simple_imputer',
                                                      SimpleImputer(fill_value=0,
                                                                    strategy='constant')),
                                                     ('passthrough',
                                                      Passthrough()),
                                                     ('model',
                                                      LinearRegression())]))

In [11]:
RMSE=[]
coefficients=[]
feature_names=None

In [12]:
kf=KFold(n_splits=10,random_state=30,shuffle=True)
for train_index, test_index in kf.split(X_copy.copy()):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    lr.fit(X_train,y_train)
    ypred=lr.predict(X_test)
    RMSE.append(mean_squared_error(y_test,ypred,squared=False))
#     display(lr.regressor_['model'].coef_)
    coefficients.append(lr.regressor_['model'].coef_)
    feature_names=list(lr.regressor_['display'].get_feature_names())

KeyError: '[183, 960, 1628, 1885] not in index'

In [ ]:
np.mean(RMSE)

Ok, this is slightly better!

In [ ]:
dfcoeff=pd.DataFrame(coefficients,columns=feature_names)

In [ ]:
features_to_keep=list(dfcoeff.mean().loc[lambda x:np.abs(x)>0.01].index)
features_to_keep

Let's look at coefficient.

### Try a submission

In [ ]:
#Instantiate a copy of Tester helper class
tester=Tester("../data/test.csv")

In [ ]:
#obtain a copy of the test set
df_test=tester.get_test_df()

In [ ]:
#reset the df
X=X_copy.copy()

#instantiate and train the model
lr=create_regressor(LinearRegression())
lr.fit(X,y)

In [ ]:
#make prediction
ypred=lr.predict(df_test)

In [ ]:
tester.write_submission(ypred,"../data/02_all_features_submission.csv")